## 1. Introduction

Ce notebook a pour objectif de tester la Stack Exchange API afin d’évaluer sa pertinence pour récupérer des données Stack Overflow.  
Nous effectuons une requête ciblée :  
- sur 50 questions contenant le tag "python",  
- avec un score supérieur à 50,  
- sur une période définie.  

Seules les données strictement nécessaires sont collectées, conformément au RGPD.



## 2. Imports et configuration

- Import des bibliothèques nécessaires (`stackapi`, `pandas`, etc.)
- Définition des paramètres de requête (site, tag, score, période)



## 3. Requête API : récupération de 50 questions "python" avec score > 50

Dans cette section, nous effectuons une requête à l’API Stack Exchange via le wrapper Python `StackAPI`.  
L’objectif est de récupérer un échantillon de 50 questions Stack Overflow contenant le tag `"python"`, ayant obtenu un score supérieur à 50, et publiées entre 2020 et 2025.

Nous filtrons également les questions ayant reçu au moins une réponse, et nous sélectionnons uniquement les champs nécessaires à l’analyse :  
- `title` : titre de la question  
- `body` : contenu principal  
- `tags` : liste des tags associés  
- `score`, `view_count`, `answer_count` : métriques d’engagement  
- `creation_date` : date de publication

Cette requête respecte les principes du RGPD en ne collectant aucune donnée personnelle.


In [3]:
# %%
from stackapi import StackAPI
import pandas as pd
from datetime import datetime

# Initialisation de l'API Stack Overflow avec clé d'application
APP_KEY = 'rl_EFmre3MifHKmYDtm5Xu24UnBj'
SITE = StackAPI('stackoverflow', key=APP_KEY)
SITE.page_size = 50  # On limite à 50 résultats comme demandé
SITE.max_pages = 1   # Une seule page

# Définir la période (ex : 2020 à 2025)
fromdate = int(datetime(2020, 1, 1).timestamp())
todate = int(datetime(2025, 1, 1).timestamp())

# Requête : questions avec le tag 'python', score > 50, au moins une réponse
questions = SITE.fetch(
    'questions',
    sort='votes',
    tagged='python',
    min=50,
    answers=1,
    fromdate=fromdate,
    todate=todate,
    filter='withbody'
)

# Conversion en DataFrame
df_api = pd.DataFrame(questions['items'])

# Sélection des colonnes utiles
df_api = df_api[[
    'question_id', 'title', 'body', 'score',
    'view_count', 'answer_count', 'tags', 'creation_date'
]]

# Suppression des doublons éventuels
df_api.drop_duplicates(subset='question_id', inplace=True)

# Conversion de la date (timestamp → datetime)
df_api['creation_date'] = pd.to_datetime(df_api['creation_date'], unit='s')




## 4. Aperçu des résultats

- Affichage des résultats
- Vérification de la structure et de la qualité des données

In [ ]:
# Affichage
display(df_api)
df_api.to_parquet("data/processed/questions_api_python_50+.parquet", index=False)


,question_id,title,body,score,view_count,answer_count,tags,creation_date
0,75608323,How do I solve &quot;error: externally-managed...,<p>When I run <code>pip install xyz</code> on ...,934,1865738,44,"[python, pip, debian, failed-installation]",2023-03-01 19:52:19
1,62983756,What is pyproject.toml file for?,<h3>Background</h3>\n<p>I was about to try Pyt...,419,435756,5,"[python, pip, packaging, pyproject.toml]",2020-07-19 17:51:19
2,75956209,Error &quot;&#39;DataFrame&#39; object has no ...,<p>I am trying to append a dictionary to a Dat...,324,730320,4,"[python, pandas, dataframe, concatenation, att...",2023-04-07 07:05:59
3,65254535,xlrd.biffh.XLRDError: Excel xlsx file; not sup...,<p>I am trying to read a macro-enabled Excel w...,299,631878,2,"[python, pandas, xlrd, pcf]",2020-12-11 15:53:36
4,60018578,What does model.eval() do in pytorch?,<p>When should I use <code>.eval()</code>? I u...,298,353091,6,"[python, machine-learning, deep-learning, pyto...",2020-02-01 15:58:15
5,71591971,How can I fix the &quot;zsh: command not found...,"<p>Since I got the <a href=""https://en.wikiped...",268,694281,22,"[python, macos, terminal, atom-editor, macos-m...",2022-03-23 18:02:02
6,64952238,docker.errors.DockerException: Error while fet...,<p>I want to install this module but there is ...,268,478152,31,"[python, linux, docker, docker-compose]",2020-11-22 08:53:15
7,62688256,sqlalchemy.exc.NoSuchModuleError: Can&#39;t lo...,<p>I'm trying to connect to a Postgres databas...,257,221298,8,"[python, postgresql, sqlalchemy, flask-sqlalch...",2020-07-02 02:56:46
8,65348890,Python was not found; run without arguments to...,"<p>I was trying to download a GUI, but the ter...",254,1846530,25,"[python, python-3.x, windows-10]",2020-12-17 21:55:37
9,66060487,"ValueError: numpy.ndarray size changed, may in...",<p>Importing from <code>pyxdameraulevenshtein<...,239,392019,20,"[python, pandas, numpy, scikit-learn, python-3.7]",2021-02-05 09:11:48


## 5. Retour d’expérience sur l’utilisation de l’API

- Facilité d’utilisation de l’API
- Qualité de la documentation
- Limites éventuelles (pagination, quota, champs disponibles)
- Respect du RGPD (aucune donnée personnelle collectée)

L’utilisation de l’API Stack Exchange via le wrapper `StackAPI` s’est révélée simple et bien documentée.  
Voici quelques observations :

- ✅ La documentation officielle est claire, avec des exemples concrets et des filtres personnalisables.
- ✅ Le wrapper `StackAPI` facilite grandement la gestion des paramètres, de la pagination et du formatage des résultats.
- ✅ L’ajout d’une clé d’application permet d’augmenter le quota de requêtes autorisées.
- ⚠️ Le filtrage par score, tag et période fonctionne bien, mais nécessite une bonne compréhension des paramètres `min`, `fromdate`, `todate`, etc.
- ⚠️ Le filtre `withbody` est utile pour récupérer le contenu complet des questions, mais peut ralentir la requête si le volume est élevé.

En résumé, l’API est tout à fait exploitable pour des extractions ciblées, à condition de bien maîtriser les filtres et les limites de pagination.  
Elle peut être utilisée pour enrichir un corpus ou tester des cas spécifiques, mais ne remplace pas une extraction massive via SEDE pour des volumes importants.


## 6. Conclusion

- L’API est-elle exploitable pour des cas plus larges ?
- Recommandations pour une utilisation future

Ce test de l’API Stack Exchange a permis de valider sa pertinence pour récupérer des questions Stack Overflow de manière ciblée.  
Nous avons pu extraire un échantillon de 50 questions contenant le tag `"python"`, avec un score élevé, sur une période définie, et sans collecter de données personnelles.

L’API s’est révélée :
- Simple à utiliser grâce au wrapper `StackAPI`
- Bien documentée
- Suffisamment flexible pour filtrer par score, tag, date, etc.

Cependant, pour des extractions massives ou des analyses globales du corpus, l’interface **Stack Exchange Data Explorer (SEDE)** reste plus adaptée.  
L’API peut être utilisée en complément pour :
- Enrichir un jeu de données existant
- Tester des cas spécifiques
- Mettre à jour un corpus avec des questions récentes

✅ **Recommandation** : conserver l’API comme outil secondaire pour des extractions ponctuelles ou dynamiques, mais privilégier SEDE pour la constitution du corpus principal.

